---
title: "Unsupervised Learning"
format:
    html: 
        code-fold: false
---

{{< embed overview.qmd >}} 

# Code 

Some of the text in these sections was informed by ChatGPT to help explain the code using prose to ensure someone not fluent in code could understand it. @gpt4o_prose_explain
